## 2. Direct Search 

In [1]:
# Import libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Import workbench libraries and the model itself
from ema_workbench import (Model, CategoricalParameter, ScalarOutcome, IntegerParameter, RealParameter)
from ema_workbench import (MultiprocessingEvaluator, Policy, Scenario, SequentialEvaluator)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, utilities
from ema_workbench import save_results
from ema_workbench import load_results

from problem_formulation_new import get_model_for_problem_formulation
from dike_model_function import DikeNetwork 

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [2]:
# Load in the results from the CSV we had saved from running the experiments
results = utilities.load_results('results/5000_scenarios_base_case.csv')
experiments, outcomes = results

# We create a 'combined' dataframe, in which the experiments and outcomes are merged within one pandas dataframe.
df_outcomes = pd.DataFrame(outcomes)
combined = pd.concat([experiments,df_outcomes],axis=1,sort=False)

combined.head()

[MainProcess/INFO] results loaded succesfully from C:\Users\ASUS\PycharmProjects\EPA1361\EPA1361_final\results\5000_scenarios_base_case.csv


A.0_ID flood wave shape    A.1_Bmax A.1_Brate  A.1_pfail    A.2_Bmax  \
0                     82.0  112.588522      10.0   0.060112  207.259283   
1                     14.0   51.029922       1.0   0.277459  302.598285   
2                    128.0   56.198749       1.5   0.880576  136.816986   
3                     96.0  123.505345       1.5   0.797417  322.362756   
4                     52.0  311.328249      10.0   0.433875  130.977495   

  A.2_Brate  A.2_pfail    A.3_Bmax A.3_Brate  A.3_pfail  ...  \
0      10.0   0.681905  298.346650      10.0   0.204375  ...   
1       1.5   0.960055  156.916590      10.0   0.004221  ...   
2      10.0   0.344347  221.692894      10.0   0.175895  ...   
3       1.5   0.977793  292.806136      10.0   0.765065  ...   
4       1.0   0.646960   95.169589      10.0   0.962059  ...   

   A.3_Dike Investment Costs 2 A.3_Expected Number of Deaths 2  \
0                            0                        0.208533   
1                            0                        1.197254   
2                            0                        0.870198   
3                            0                        0.050221   
4                            0                        0.000000   

   A.4_Expected Annual Damage 2  A.4_Dike Investment Costs 2  \
0                  0.000000e+00                            0   
1                  0.000000e+00                            0   
2                  0.000000e+00                            0   
3                  2.064967e+07                            0   
4                  3.580277e+07                            0   

  A.4_Expected Number of Deaths 2  A.5_Expected Annual Damage 2  \
0                        0.000000                           0.0   
1                        0.000000                           0.0   
2                        0.000000                           0.0   
3                        0.011789                           0.0   
4                        0.018065                           0.0   

  A.5_Dike Investment Costs 2 A.5_Expected Number of Deaths 2  \
0                           0                             0.0   
1                           0                             0.0   
2                           0                             0.0   
3                           0                             0.0   
4                           0                             0.0   

  RfR Total Costs 2  Expected Evacuation Costs 2  
0               0.0                          0.0  
1               0.0                          0.0  
2               0.0                          0.0  
3               0.0                          0.0  
4               0.0                          0.0  

[5 rows x 104 columns]

In [ ]:
# The following function is used to aggregate the outcomes into the KPI we want
# This iterates over all the locations and round numbers, and creates a new column summing the values per location and round.
# If we want to aggregate over the location, "aggregate" equals "location" and therefore the KPI is added per location and not in total.
# If we want to aggregate over the province, "aggregate" equals "name of the province" and therefore the KPI is added per province.
# On the contrary, if aggregate equals "total", the total value is appended to the dataframe.
def aggregate_kpi(data, kpi, aggregate):
    locations = ["A.1", "A.2", "A.3", "A.4", "A.5"]
    provinces = ["Overijssel", "Gelderland"]
    kpi_columns = []
    
    if kpi == "RfR Total Costs" or kpi == "Expected Evacuation Costs":
        kpi_columns.append(kpi + " 0")
        kpi_columns.append(kpi + " 1")
        kpi_columns.append(kpi + " 2")
        
        data[kpi] = data[kpi_columns].sum(axis=1)
    else:
        if aggregate == "total":
            for location in locations:
                kpi_columns.append(location + "_" + kpi + " 0")
                kpi_columns.append(location + "_" + kpi + " 1")
                kpi_columns.append(location + "_" + kpi + " 2")

            data[kpi] = data[kpi_columns].sum(axis=1)
            
        elif aggregate == "province":
            for province in provinces:
                if province == 'Overijssel':
                    kpi_columns = []
                    kpi_columns.append("A.4" + "_" + kpi + " 0")
                    kpi_columns.append("A.4" + "_" + kpi + " 1")
                    kpi_columns.append("A.4" + "_" + kpi + " 2")
                    kpi_columns.append("A.5" + "_" + kpi + " 0")
                    kpi_columns.append("A.5" + "_" + kpi + " 1")
                    kpi_columns.append("A.5" + "_" + kpi + " 2")                    
                    
                    data["Overijssel" + "_" + kpi] = data[kpi_columns].sum(axis=1)
      
           
                else:
                    kpi_columns = []
                    kpi_columns.append("A.1" + "_" + kpi + " 0")
                    kpi_columns.append("A.1" + "_" + kpi + " 1")
                    kpi_columns.append("A.1" + "_" + kpi + " 2")
                    kpi_columns.append("A.2" + "_" + kpi + " 0")
                    kpi_columns.append("A.2" + "_" + kpi + " 1")
                    kpi_columns.append("A.2" + "_" + kpi + " 2") 
                    kpi_columns.append("A.3" + "_" + kpi + " 0")
                    kpi_columns.append("A.3" + "_" + kpi + " 1")
                    kpi_columns.append("A.3" + "_" + kpi + " 2")                    

                    data["Gelderland" + "_" + kpi] = data[kpi_columns].sum(axis=1)
            
        else:
            for location in locations:
                kpi_columns = []
                kpi_columns.append(location + "_" + kpi + " 0")
                kpi_columns.append(location + "_" + kpi + " 1")
                kpi_columns.append(location + "_" + kpi + " 2")

                data[location + "_" + kpi] = data[kpi_columns].sum(axis=1)
                
    return data

# Append the KPIs we would like to analyse.
combined = aggregate_kpi(combined, "Expected Number of Deaths", "total")
combined = aggregate_kpi(combined, "RfR Total Costs", "total")
combined = aggregate_kpi(combined, "Expected Evacuation Costs", "total")
combined = aggregate_kpi(combined, "Expected Annual Damage", "province")
combined = aggregate_kpi(combined, "Dike Investment Costs", "province")

By executing a scenario discovery analysis, we are planning to find uncertain futures where the resulting values are undesirable, given that no policy is implemented yet. 

To find scenarios in which these outcomes perform poorly, the Patient Rule Induction Method (PRIM) is used, which is incorporated in the EMA workbench. Originally designed by Friedman & Fisher (1999), PRIM is a technique that iteratively narrows down the uncertainty space until boxes are found that form a good trade-off between coverage (what fraction of the total outcomes of interest are in the box) and density (what fraction of all cases in the box are actually of interest).

**Undesirability** - to be updated

Undesirability is in this case defined as being in the 66.6th percentile of outcomes, meaning that scenarios in which deaths, damage and variance of damage between locations are all in the upper 33.3% of their range are sought.  

**Results** - to be uodated with our results

From the PRIM follows that 879 scenarios are of interest within the scenario space of 5000 (=17.5%). These 879 scenarios can be mainly credited to four uncertainties. The uncertainty ranges that creates the worst scenarios are the following:

- A.1_pfail between 0 and 0.34
- A.2_pfail between 0 and 0.96
- A.3_pfail between 0 and 0.91
- A.1_Bmax between 0 and 190

In [ ]:
from ema_workbench.analysis import prim

# Select the uncertainty values
cleaned_experiments = combined.iloc[:,:19]

# Calculate the value (threshold) where the upper 33.3% range starts
percentile_costs = np.percentile(combined["Expected Annual Damage"], 66)
percentile_deaths = np.percentile(combined["Expected Number of Deaths"], 66)
percentile_var = np.percentile(combined["Variance Expected Annual Damage"], 66)

# Bool the values if they are larger than the threshold
combined["Cost_bool"] = combined["Expected Annual Damage"] > percentile_costs
combined["Death_bool"] = combined["Expected Number of Deaths"] > percentile_deaths
combined["Var_bool"] = combined["Variance Expected Annual Damage"] > percentile_var

# If cost and deaths and variance arein the 66th upper percentiles indicate true
y = combined["Cost_bool"] & combined["Death_bool"] & combined["Var_bool"]

# Execute prim algorithm
prim_alg = prim.Prim(cleaned_experiments, y, threshold=0.6)
box1 = prim_alg.find_box()

# Show trade off plot
box1.show_tradeoff()
plt.show()

It is important that a good box is selected, that properly trades off density and coverage. Whereas ideally, the density should be around 0.8, this would in this case lead to a coverage of less than 0.3. Therefore, a box has been chosen with a slightly lower density of 0.696, so that coverage is still somewhat representative. The fact that this includes 4 dimensions instead of 5 might be too limiting, but it does make interpretation easier. 

In [ ]:
# Choosing the best box and inspecting the corresponding coverage, density and uncertainty ranges.
box1.inspect(38, style='graph')
box1.select(38)
plt.show()

The tradeoffs can be plotted using dimensional stacking and through a matrix of scatter plots. For the report, the latter was chosen, because it feels more easily interpretable in this case. 

In [ ]:
from ema_workbench.analysis import dimensional_stacking

dimensional_stacking.create_pivot_plot(cleaned_experiments, y, nr_levels=3)
plt.show()

In [ ]:
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.savefig("images/PRIM_results_distributionmatrix.png")
plt.show()

In [ ]:
This finalizes the exploration section of this analysis, which are fully reported and interpreted in section 4.1 and 4.2 of the main report.